# Mision To Mars

In [1]:
%reload_ext lab_black

In [2]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Reflect Tables into SQLAlchemy ORM

In [36]:
# Define NASA News url
BASE_URL = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

driver = webdriver.Firefox()
driver.get(BASE_URL)
driver.implicitly_wait(10)
html = driver.page_source
driver.close()

#Loading page
soup = BeautifulSoup(html, "html.parser")

In [45]:
# Extract titles and text from divs
news_title = [
    tag.text
    for tag in [li for ul in soup for li in ul.findAll("div", class_="content_title")]
]

news_p = [
    tag.text
    for tag in [
        li for ul in soup for li in ul.findAll("div", class_="article_teaser_body")
    ]
]

div_list = list(zip(news_title, news_p))
div_list

[("A Year of Surprising Science From NASA's InSight Mars Mission",
  "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."),
 ("Mars InSight Lander to Push on Top of the 'Mole'",
  'Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.'),
 ('NASA Adds Return Sample Scientists to Mars 2020 Leadership Team',
  'The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.'),
 ("NASA's Mars Reconnaissance Orbiter Undergoes Memory Update",
  'Other orbiters will continue relaying data from Mars surface missions for a two-week period.'),
 ("NASA's Mars 2020 Rover Goes Coast-to-Coast to Prep for Launch",
  "The agency's first step in returning rocks from Mars just arrived at Kennedy Space Center. The Mars 2020 team now begins readying for a launch to the Red Planet this July

In [ ]:
# _li = soup.find('li', {'class':'list'})
# results = [[i.text for i in b.find_all('li')] for b in _div.find_all('ul')]

# results

# div_dict = {
#     [
#     li
#         for ul in soup
#         for li in ul.findAll(
#             "div", class_="content_title"
#         )
#     ]
#     ,
#     [
#     li
#         for ul in soup
#         for li in ul.findAll(
#             "div", {"class": "article_teaser_body"}
#         )
#     ]
# }

# div_p = [li for ul in soup for li in ul.findAll("div", class_="article_teaser_body")]
